In [2]:
import pandas as pd
import cPickle as pickle
from numpy import isnan
from nltk.corpus import stopwords

from textblob import Word
from textblob.base import BaseTokenizer
from textblob import TextBlob
from textblob import Blobber
from textblob.tokenizers import WordTokenizer

In [3]:
data=pd.read_csv("Assignment 3 Edmunds Posts.csv", usecols=[0])

In [4]:
data['Posts'] = data['Posts'].map(lambda x: x.replace("LexusES","ES"))
data['Posts'] = data['Posts'].map(lambda x: x.replace("LexusLS","LS"))
data['Posts'] = data['Posts'].map(lambda x: x.replace("LexusRX","RX"))

In [5]:
sw=stopwords.words('english')

sw.remove('not')

models_skinny=["es","ls","rx","a8","a6","3series",\
               "5series","7series","xj","sclass"] 

In [9]:
models=[" "+x+" " for x in models_skinny]
models

[' es ',
 ' ls ',
 ' rx ',
 ' a8 ',
 ' a6 ',
 ' 3series ',
 ' 5series ',
 ' 7series ',
 ' xj ',
 ' sclass ']

In [6]:
def tKnzr (xstring):
    global sw
    tokens=list(TextBlob(str(xstring)).words)
    removeStopWords=[word for word in tokens if word.lower() not in sw]
    lemmaed=[Word(w).lemmatize() for w in removeStopWords]
    lowercase=[word.lower() for word in lemmaed]
    return lowercase

In [7]:
def finder (tokensList, modelList, numberWords):
    blanklist=[]
    for i in xrange(len(tokensList)):
        if tokensList[i] in modelList:
            blanklist.append(tokensList[i-numberWords:i+1+numberWords])
    return blanklist

In [ ]:
#filling in the data matrix

for index,l in enumerate(df['model_strings']):
    for string in l: #looping over all the strings in model_strings
            for model in models: #loop over all the models in model list
                if model in string: #check if model is in a particular list
                    if isnan(df[model].iloc[index]): #correcting for neutral
                        df[model].iloc[index]=0
                    df[model].iloc[index]+=tb(string).sentiment[0]  

In [18]:
def sentiment_compiler(postseries,modelsList,numberofwords):
    newseries=postseries.apply(tKnzr)
    newseries=newseries.apply(lambda x: finder(x,modelsList, numberofwords))
    newseries=newseries.apply(lambda x: [' '.join(inner) for inner in x])
    models=[" "+x+" " for x in modelsList]
    df=pd.DataFrame(columns=modelsList).join(newseries, how="outer")
    for index,l in enumerate(newseries):
        for string in l: #looping over all the strings in model_strings
                for model, model_skinny in zip(models,modelsList): #loop over all the models in model list
                    if model in string: #check if model is in a particular list
                        if isnan(df[model_skinny].iloc[index]): #correcting for neutral
                            df[model_skinny].iloc[index]=0
                        df[model_skinny].iloc[index]+=tb(string).sentiment[0]  

In [19]:
sentiment_compiler(data['Posts'],models_skinny,2)

NameError: global name 'tb' is not defined